In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest
df = sns.load_dataset("tips")
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [2]:
df.groupby("smoker")["total_bill"].mean() # fark şans eseri mi değil mi

smoker
Yes    20.756344
No     19.188278
Name: total_bill, dtype: float64

In [3]:
df.loc[df["day"].isin(["Sat","Thur"]),"total_bill"]

19     20.65
20     17.92
21     20.29
22     15.77
23     39.42
       ...  
239    29.03
240    27.18
241    22.67
242    17.82
243    18.78
Name: total_bill, Length: 149, dtype: float64

In [4]:
#normal dağılım testi shapiro h0=normal dağılım var
test_stat, pvalue = shapiro(df.loc[df["smoker"]=="Yes","total_bill"])
print(f"test_stat {test_stat:.4f} pvalue {pvalue:.4f}")
# p <0.05 ise h0 red yani normal dağılım değil

test_stat 0.9367 pvalue 0.0002


In [5]:
#normal dağılım testi shapiro h0 = normal dağılım var
test_stat, pvalue = shapiro(df.loc[df["smoker"]=="No","total_bill"])
print(f"test_stat {test_stat:.4f} pvalue {pvalue:.4f}")
# p <0.05 ise h0 red yani normal dağılım değil

test_stat 0.9045 pvalue 0.0000


In [6]:
# varyans homojenliği varsayımı h0 = varyans homojendir. levene testi
test_stat, pvalue = levene(df.loc[df["smoker"]=="Yes","total_bill"],
                           df.loc[df["smoker"]=="No","total_bill"])
print(f"test_stat {test_stat:.4f} pvalue {pvalue:.4f}")
# p < 0.05 ise h0 red yani varyanslar homojen değil

test_stat 4.0537 pvalue 0.0452


In [7]:
#varsayımlar sağlanıyorsa t testi (parametrik) sadece varyans sağlanmıyorsa parametre belirt equal_var=False gir
#varsayımlar sağlanmıyorsa mannwhitneyyu (non- parametrik)

In [8]:
# sağlanmış senaryosunda h0 = aralarında fark yok
test_stat, pvalue = ttest_ind(df.loc[df["smoker"]=="Yes","total_bill"],
                                df.loc[df["smoker"]=="No","total_bill"],
                                equal_var = True)
print(f"test_stat {test_stat:.4f} pvalue {pvalue:.4f}")

# p > 0.05 yani p büyük h0 reddedilemez yani h0 var ikisi aynı fark şans eseri oluşmuş

test_stat 1.3384 pvalue 0.1820


In [9]:
# varsayımlar sağlanmıyorsa h0 = aralarında fark yok

test_stat, pvalue = mannwhitneyu(df.loc[df["smoker"]=="Yes","total_bill"],
                                df.loc[df["smoker"]=="No","total_bill"]
                                )
print(f"test_stat {test_stat:.4f} pvalue {pvalue:.4f}")
# p > 0.05 yani h0 reddedilemez yani h0 sağlanır aralarında fark yok 

test_stat 7531.5000 pvalue 0.3413


In [10]:
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [11]:
df.groupby("sex")["age"].mean()

sex
female    27.915709
male      30.726645
Name: age, dtype: float64

In [12]:
#h0 anlamlı fark yoktur yani eşittir p > 0.05 ise h0 geçerli olur 

In [13]:
# normal dağılım kontrolü
test_stat, pvalue = shapiro(df.loc[df["sex"]=="female","age"].dropna())
print(f"test stat {test_stat:.4f}, pvalue {pvalue:.4f}")
# p <0.05 ise h0 red yani normal dağılım yok

test stat 0.9848, pvalue 0.0071


In [14]:
# normal dağılım kontrolü
test_stat, pvalue = shapiro(df.loc[df["sex"]=="male","age"].dropna())
print(f"test stat {test_stat:.4f}, pvalue {pvalue:.4f}")
# p <0.05 ise h0 red yani normal dağılım yok

test stat 0.9747, pvalue 0.0000


In [15]:
#varyans kontrolü
test_stat, pvalue = levene(df.loc[df["sex"]=="female","age"].dropna(),
                           df.loc[df["sex"]=="male","age"].dropna())
print(f"test stat {test_stat:.4f}, pvalue {pvalue:.4f}")

# p > 0.05 yani h0 geçerli varyanslar homojen

test stat 0.0013, pvalue 0.9712


In [23]:
# non-parametrik test
test_stat, pvalue = mannwhitneyu(df.loc[df["sex"]=="female","age"].dropna(),
                                 df.loc[df["sex"]=="male","age"].dropna())
print(f"test stat {test_stat:.4f}, pvalue {pvalue:.4f}")

# p < 0.05 h0 red, yaş ortalamaları arasında istatistiki olarak fark var

test stat 53212.5000, pvalue 0.0261


In [24]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [26]:
df.groupby("Outcome")["Age"].mean()

Outcome
0    31.190000
1    37.067164
Name: Age, dtype: float64

In [28]:
#normallik varsayımı incele h0 sağlanır

test_stat, pvalue = shapiro(df.loc[df["Outcome"]==1,"Age"].dropna())
print(f" test stat {test_stat:.4f}, pvalue {pvalue:.4f} ")

# p <0.05 h0 red yani normallik yok

 test stat 0.9546, pvalue 0.0000 


In [29]:
#normallik varsayımı incele h0 sağlanır

test_stat, pvalue = shapiro(df.loc[df["Outcome"]==0,"Age"].dropna())
print(f" test stat {test_stat:.4f}, pvalue {pvalue:.4f} ")

# p <0.05 h0 red yani normallik yok

 test stat 0.8012, pvalue 0.0000 


In [36]:
#normallik sağlanmadığı için nonparametrik medyan kıyaslaması olarak da geçer

test_stat, pvalue = mannwhitneyu(df.loc[df["Outcome"]==1,"Age"].dropna(),
                                 df.loc[df["Outcome"]==0,"Age"].dropna())
print(f" test stat {test_stat:.4f}, pvalue {pvalue:.4f} ")
# p < 0.05 h0 red ikisi eşit değil fark var

 test stat 92050.0000, pvalue 0.0000 
